In [ ]:
import pandas as pd
import os
import numpy as np
from scipy import signal

def filter_signal(dat:np.ndarray, 
                  band=[1,40], 
                  sampling_rate:int=500):
    # data: 1D array
    [b, a] = signal.iirnotch(60, 30, sampling_rate) # power noise
    dat = signal.filtfilt(b, a, dat)
    [b, a] = signal.butter(3, band, 'bandpass', fs=sampling_rate)
    dat = signal.filtfilt(b, a, dat)
    # dat =  np.convolve(dat, np.ones(sampling_rate//20) / (sampling_rate//20), mode='same')
    return dat

def organize_emotion_data(easy_, resample_fs=125):
    sig = np.zeros((9,10,500*120))
    easy = easy_[:,1::2]
    event = np.where(easy[8]!=0)[0]
    if len(event) < 2:
        easy = easy_[:,0::2]
        event = np.where(easy[8]!=0)[0]
    if len(event) != 9:
        event = event[1:]
    event = np.stack((event, easy[8,event]), 1)
    easy = filter_signal(easy)
    sig[0] = np.concatenate((easy[:,event[0,0]:event[0,0]+500*60], easy[:,event[0,0]+500*65:event[0,0]+500*125]), axis=1)
    for num_trial in range(1, 9):
        sig[num_trial] = easy[:,event[num_trial,0]:event[num_trial,0]+500*120]
    sig = signal.resample(sig, resample_fs*120, axis=2)
    return sig[:,:8] #, event[:,1]

def load_emotion_data():
    ROOT_PATH = r'D:/One_한양대학교/private object minsu/coding/data/samsung_2024/Day 1/'
    fs = 125
    subj_list = os.listdir(ROOT_PATH)
    print(subj_list)

    emotion_data = np.zeros((len(subj_list), 9, 8, fs*120))
    emotion_label = np.zeros((len(subj_list), 9, 2))
    for idx, subj in enumerate(subj_list):
        dat_list = os.listdir(ROOT_PATH+subj)
        # print(subj, dat_list[0])

        easy = np.asarray(pd.read_csv(ROOT_PATH + subj + '/' + dat_list[0], delimiter='\t')).T
        emotion_data[idx] = organize_emotion_data(easy, fs)

        label_raw = pd.read_csv(ROOT_PATH + subj + '/' + [s for s in dat_list if 'Emotion' in s][0])
        label_np = np.array(label_raw[['resting','H1','H2','P1','P2','A1','A2','S1','S2']])
        label_np[2,0] = -1
        emotion_label[idx] = label_np[:2, np.argsort(label_np[2])].T
    return emotion_data[:, np.argsort(label_np[2])], emotion_label

emotion_data, emotion_label = load_emotion_data()

['02.YBJ', '03.JHS', '04.HJH', '05.LHS', '06,KAY(W)', '07.HSY', '08.KHJ(W)', '09.LGY(W)', '10.PJU', '11.KMJ(W)', '12.KJG(W)', '13.LHJ', '14.YSA', '15.KHI(W)', '16.CHJ(W)', '17.PCG', '18.ASJ(W)', '19.RWO', '20.CYS', '21.HDE(W)', '22.SMS', '23.LHJ', '24.OJM', '25.SJH(W)', '26.HYJ(W)', '27.KTH(W)', '28.JMY(W)', '29.SSJ', '30.KHW', '31.PYJ(W)', '32.JMK(W)', '33.CYR']


In [2]:
np.save('emotion_data.npy',emotion_data)
np.save('emotion_label.npy',emotion_label)

In [3]:
"""
emotion_data: shape (33, 9, 8, 15000) = (num_subject, trials, electrode channels, time samples)
    trials (9 trials): resting, happy/peaceful, sad/angry 
    electrode channels (8 channels): Fp(AF7, FPZ, AF8), Central (C3, CZ, C4), Ear (Left, Right) 
    time samples: 125 Hz downsampled, 1-40 Hz bandpass filtered, 60 Hz notch filtered

NOTE: signal for resting state is concatenated signal (60 sec eyes open + 60 sec eyes closed)

emotion_label: shape (33, 9, 2) = (num_subject, trials, emotional state)
    emotional state: idx 0 is for arousal, and idx 1 is for valence. scored 1 to 5.

NOTE: you can transform score label into binary label (pos/neg valence and high/low arousal), as follows:
        new_emotion_label = np.array(emotion_label>2,int)
""" 

'\nemotion_data: shape (33, 9, 8, 15000) = (num_subject, trials, electrode channels, time samples)\n    trials (9 trials): resting, happy/peaceful, sad/angry \n    electrode channels (8 channels): Fp(AF7, FPZ, AF8), Central (C3, CZ, C4), Ear (Left, Right) \n    time samples: 125 Hz downsampled, 1-40 Hz bandpass filtered, 60 Hz notch filtered\n\nNOTE: signal for resting state is concatenated signal (60 sec eyes open + 60 sec eyes closed)\n\nemotion_label: shape (33, 9, 2) = (num_subject, trials, emotional state)\n    emotional state: idx 0 is for arousal, and idx 1 is for valence. scored 1 to 5.\n\nNOTE: you can transform score label into binary label (pos/neg valence and high/low arousal), as follows:\n        new_emotion_label = np.array(emotion_label>2,int)\n'